# Numerical Integrals to obtain $\sigma_*$

After one solved the Jeans equation to obtain $\sigma^2_r$ one maight want to compare it to observacional results, and in order to do it, it is necessary to solve the follow equation

$$
\left \langle  \sigma^2_*\right \rangle = \frac{2\int_0^{\infty}\int_R^{\infty}\omega(R)\frac{\nu(r)\sigma^2_r(r)}{\sqrt{r^2 - R^2}}\left (1 - \beta\frac{R^2}{r^2}\right )rdrRdR}{\int_0^{\infty}\int_R^{\infty}\omega(R)\frac{2\nu(r)}{\sqrt{r^2 - R^2}}rdrRdR}
$$

where $\nu(r)$ is the light profile, $\left (1 - \beta\frac{R^2}{r^2}\right )$ is the anysotropy term and $\omega(R)$ is the weight function that takes into account the observational conditions, for SDSS spectra it can be given by equation 21 in [arXiv:0907.4992v2](https://arxiv.org/pdf/0907.4992.pdf). Finally the velocity dispersion measured through the spectra is given by $\sigma_* = \sqrt{\left \langle  \sigma^2_*\right \rangle}$.

Nevertheless, for a constant $\beta$ this equation can be solved analitically, for more conplex anisotropies ($\beta = \beta(r))$ it became a trick one to solve. 
In this Notebook I inted to test numerical integration metods to solve this equation, using a constant $\beta$ because of the possibility of cross check the numerical results with the analitical ones.


In [91]:
### libraries
import sys

sys.path.append('../')

import Velocity_dispersion
import rings2cosmo
import pandas as pd
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
import seaborn

from astropy import constants as const
from astropy import units as u
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0 = 67.3, Om0 = 0.315)

## The data
I wil use the same data as in [arXiv:1701.00357](https://arxiv.org/pdf/1701.00357.pdf).

In [2]:
## importing cao object for tests
data = pd.read_csv('../data/sample80.csv')
data.head()

,name,zl,zs,sigma_0,dsigma_0,theta_E,theta_ap,sigma_atm,sigma_ap,d_sigma_ap
0,J2321-0939,0.082,0.532,246,8,1.60,1.5,1.4,249,8
1,J1106+5228,0.096,0.407,268,13,1.23,1.5,1.4,262,13
2,J1143-0144,0.106,0.402,264,13,1.68,1.5,1.4,269,13
3,J0841+3824,0.116,0.657,222,11,1.41,1.5,1.4,225,11
4,J0044+0113,0.120,0.196,267,13,0.79,1.5,1.4,266,13


In [3]:
### variables and unity convesion

z_L = data['zl'][0]
z_S = data['zs'][0]
theta_E = (data['theta_E'].values * u.arcsec).to(u.rad)[0].value
theta_ap = (data['theta_ap'].values * u.arcsec).to(u.rad)[0].value
seeing_atm = (data['sigma_atm'].values * u.arcsec).to(u.rad)[0].value
velDisp = data['sigma_ap'][0]
velDispErr = data['d_sigma_ap'][0]

In [4]:
print(z_L, z_S, theta_E ,theta_ap, seeing_atm, velDisp, velDispErr)

0.082 0.532 7.757018897752577e-06 7.27220521664304e-06 6.787391535533504e-06 249 8


## The numerical integral

Here I am using the function scipy.integrate.dbquad (see in Velocity_dispersion.py).

Starting with a smal test.

In [5]:
#parameters not related to data
alpha = 2.0
beta = 0.18
delta = 2.4
gamma = 1.0
epsabs = 1.49e-03

In [6]:
#the numerical result
r = Velocity_dispersion.sigma_star(z_L,z_S,theta_E, seeing_atm, theta_ap, alpha, beta, delta,gamma, epsabs)

/home/grasi/anaconda3/envs/Velocity_dispersion2022/lib/python3.10/site-packages/scipy/integrate/quadpack.py:879: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
/home/grasi/anaconda3/envs/Velocity_dispersion2022/lib/python3.10/site-packages/scipy/integrate/quadpack.py:879: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
/home/grasi/anaconda3/envs/Velocity_dispersion2022/lib/python3.10/site-packages/scipy/integrate/quadpack.py:879: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in orde

In [7]:
#(240.11096922329975, 7447.320230947961, 267175926.2984453) relative
r

(240.10889884740445, 7447.191801281745, 86574202.81956239)

In [15]:
#the analitical result
rings2cosmo.vel(z_S, z_L, theta_E, seeing_atm, theta_ap, alpha, beta, gamma, delta)

<Quantity 242.72671402>

So here we can see that the numerical integration got satisfactorily close to the analitical value (relative error ~ 1%), even though the integral error is quite righ.
So I will generate a few values in order to perform some tests

In [10]:
#this can take a wile
#about 1.5 days
results = []
cols = ("name", "numerical_sigma", "integral_result", "integral_error", "epsabs")

alpha = 2.0
beta = 0.18
delta = 2.4
gamma = 1.0
epsabs_values = (1.49e-01, 1.49e-02, 1.49e-03, 1.49e-04, 1.49e-05, 1.49e-06, 1.49e-07, 1.49e-08)
data = pd.read_csv('../data/sample80.csv')

for epsabs in epsabs_values:
    for i in range(len(data)):
        ## data
        name = data['name'][i]
        z_L = data['zl'][i]
        z_S = data['zs'][i]
        theta_E = (data['theta_E'].values * u.arcsec).to(u.rad)[i].value
        theta_ap = (data['theta_ap'].values * u.arcsec).to(u.rad)[i].value
        seeing_atm = (data['sigma_atm'].values * u.arcsec).to(u.rad)[i].value
        
        integral = Velocity_dispersion.sigma_star(z_L,z_S,theta_E, seeing_atm, theta_ap, alpha, beta, delta,gamma, epsabs)
        
        temp = (name, integral[0], integral[1],integral[2], epsabs)
        results.append (dict(zip(cols,temp)))
        


/home/grasi/anaconda3/envs/Velocity_dispersion2022/lib/python3.10/site-packages/scipy/integrate/quadpack.py:879: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
/home/grasi/anaconda3/envs/Velocity_dispersion2022/lib/python3.10/site-packages/scipy/integrate/quadpack.py:879: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
/home/grasi/anaconda3/envs/Velocity_dispersion2022/lib/python3.10/site-packages/scipy/integrate/quadpack.py:879: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in orde

In [11]:
df = pd.DataFrame(results)
df.to_csv("results.csv", index = False)